<a href="https://colab.research.google.com/github/hes-0454/compile-python-/blob/main/RAG1-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 33.8 MB/s eta 0:00:00


Create Embeddings
Use a pre-trained model from the Sentence Transformers library

In [4]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample documents
documents = [
    "RAG combines retrieval and generation.",
    "It uses embeddings for document retrieval.",
    "Integration with language models improves context."
]

# Generate embeddings
embeddings = model.encode(documents, convert_to_tensor=True)
import faiss
import numpy as np

# Convert embeddings to numpy array
embeddings_np = embeddings.detach().numpy()

# Create FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)
# Example query
query = "How does RAG work?"
query_embedding = model.encode(query, convert_to_tensor=True).detach().numpy()

# Search in FAISS index
D, I = index.search(np.array([query_embedding]), k=2)  # Top-2 results
print("Top documents:", [documents[i] for i in I[0]])
from transformers import pipeline

# Load a generative language model
generator = pipeline('text-generation', model='gpt2')

# Combine query with retrieved documents
context = " ".join([documents[i] for i in I[0]])
input_text = f"Query: {query}\nContext: {context}\nAnswer:"

# Generate response
response = generator(input_text, max_length=50)
print(response[0]['generated_text'])


Top documents: ['RAG combines retrieval and generation.', 'It uses embeddings for document retrieval.']


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Query: How does RAG work?
Context: RAG combines retrieval and generation. It uses embeddings for document retrieval.
Answer: It collects a JSON URL from the RAG document, which then parses the query into response codes
